O objetivo deste notebook é obter um bom modelo de predição de uso de oferta.

Lembre-se de que:

- Não se pode utilizar dados de um mesmo cliente no treino, validação e teste ao mesmo tempo
- Pode-se utilizar modelos de arvore para, por exemplo, entregar a probabilidade de o cliente utilizar o cupom
- Um modelo por cluster pode melhorar os resultados?

# 1. Importando bibliotecas e dados

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from joblib import dump
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

# Modelos de classificação
from catboost import CatBoostClassifier

import sys
sys.path.append('../src/')

from Model import Model

In [2]:
offer_for_modeling = pd.read_csv("../data/processed/offers_for_modeling.csv")
# Target
y_col = ["TARGET_USED_OFFER"]

# Coluna dos clusters
cluster_col = "cluster"

# Coluna que separa nosso dataset dentro dos clusters
partition_col = "account_id"

# Colunas endógenas (temos controle)
endogenous_cols = [
    "offer_received_date", "channel_mobile", "channel_email", "channel_social",
    "channel_web", "discount_value", "offer_duration", "offer_min_value"
]
endogenous_cat_cols = ["offer_type"]

# Colunas exógenas (externas, não temos controle)
exogenous_cols = [
    "recently_viewed_info_offer","age","credit_card_limit","total_transactions","avg_amount",
    "most_used_offer_type","transactions_offer_rate","avg_time_to_use_offer","avg_time_to_view_offer",
    "all_coupon_usage_rate","pct_used_channel_mobile","pct_used_channel_email","pct_used_channel_social",
    "pct_used_channel_web","pct_used_type_bogo","pct_used_type_discount","pct_used_type_informational",
    "all_coupon_viewed_rate","pct_viewed_channel_mobile","pct_viewed_channel_email","pct_viewed_channel_social",
    "pct_viewed_channel_web","pct_viewed_type_bogo","pct_viewed_type_discount","pct_viewed_type_informational",
    "total_offers_received","total_mobile_offer","total_email_offer","total_social_offer",
    "total_web_offer","total_offers_bogo","total_offers_discount","total_offers_informational"
]

# Variáveis categóricas
categorical_features = [
    "age","credit_card_limit","total_transactions","avg_amount",
    "most_used_offer_type","transactions_offer_rate","avg_time_to_use_offer","avg_time_to_view_offer",
    "all_coupon_usage_rate","pct_used_channel_mobile","pct_used_channel_email","pct_used_channel_social",
    "pct_used_channel_web","pct_used_type_bogo","pct_used_type_discount","pct_used_type_informational",
    "all_coupon_viewed_rate","pct_viewed_channel_mobile","pct_viewed_channel_email","pct_viewed_channel_social",
    "pct_viewed_channel_web","pct_viewed_type_bogo","pct_viewed_type_discount","pct_viewed_type_informational",
    "total_offers_received","total_mobile_offer","total_email_offer","total_social_offer",
    "total_web_offer","total_offers_bogo","total_offers_discount","total_offers_informational",
]

In [3]:
account_cluster = pd.read_csv("../data/processed/profile_clustered_id.csv")

# Join para acrescentar cluster na tabela antes da modelagem
offer_for_modeling = (pd.merge(offer_for_modeling, account_cluster,
    on="account_id", how="left")
)

del(account_cluster)

# Aplicar one-hot encoding apenas nas colunas endógenas
df_encoded = pd.get_dummies(offer_for_modeling[endogenous_cat_cols], prefix=endogenous_cat_cols, drop_first=False, dtype=float)

# Definindo todas as variáveis do modelo
X_cols = endogenous_cols + list(df_encoded.columns) + exogenous_cols

# Concatenar com as colunas exógenas
offer_for_modeling = pd.concat(
    [
        offer_for_modeling,
        df_encoded
    ],
    axis=1
)

# Coletando todos os clusters
unique_clusters = list(offer_for_modeling[cluster_col].sort_values().unique())

# 2. Analise pré treino

## 2.1. Verificando quantidade de dados por cluster

Verificando se mesmo ao separar os dados de acordo com os clusters, temos dados suficientes para treinar modelos de ML

In [4]:
offer_for_modeling["cluster"].value_counts().sort_index() / len(X_cols)

cluster
0    320.604651
1    144.930233
2    186.418605
3    171.139535
4    798.860465
5    193.558140
6    127.372093
7    228.767442
Name: count, dtype: float64

Vemos que para todos os 8 clusters temos mais de 10 linhas para cada coluna, passando do mínimo recomendado para treino de modelos de ML

## 2.2. Removendo clusters com target constantes e balanceando targets

Considerando que alguns clusters não usam ofertas, vamos remover os que não usam

E balancear a target nos clusters que usam

In [5]:
for _, cl in enumerate(unique_clusters.copy()):
    if offer_for_modeling[offer_for_modeling["cluster"] == cl][y_col[0]].sum() == 0:
        print(f"Cluster {cl} será removido da lista de treino porque não possui uso de ofertas.")
        print()
        unique_clusters.remove(cl)
    else:
        # Printando informações para ajudar a entender o que esta acontecendo
        print(f"Distribuição da variável target para o cluster {cl}")
        target_count = offer_for_modeling[offer_for_modeling["cluster"] == cl][y_col[0]].value_counts().to_dict()
        target_dist = offer_for_modeling[offer_for_modeling["cluster"] == cl][y_col[0]].value_counts(normalize=True).to_dict()
        print(f"- Positivo: {round(target_dist[1]*100)}% ({target_count[1]:,} → {round(target_count[1] / len(X_cols))} linhas por coluna)")
        print(f"- Negativo: {round(target_dist[0]*100)}% ({target_count[0]:,} → {round(target_count[0] / len(X_cols))} linhas por coluna)")
        print()


Cluster 0 será removido da lista de treino porque não possui uso de ofertas.

Distribuição da variável target para o cluster 1
- Positivo: 92% (5,723 → 133 linhas por coluna)
- Negativo: 8% (509 → 12 linhas por coluna)

Distribuição da variável target para o cluster 2
- Positivo: 34% (2,715 → 63 linhas por coluna)
- Negativo: 66% (5,301 → 123 linhas por coluna)

Cluster 3 será removido da lista de treino porque não possui uso de ofertas.

Distribuição da variável target para o cluster 4
- Positivo: 93% (31,950 → 743 linhas por coluna)
- Negativo: 7% (2,401 → 56 linhas por coluna)

Distribuição da variável target para o cluster 5
- Positivo: 75% (6,252 → 145 linhas por coluna)
- Negativo: 25% (2,071 → 48 linhas por coluna)

Distribuição da variável target para o cluster 6
- Positivo: 31% (1,685 → 39 linhas por coluna)
- Negativo: 69% (3,792 → 88 linhas por coluna)

Distribuição da variável target para o cluster 7
- Positivo: 64% (6,328 → 147 linhas por coluna)
- Negativo: 36% (3,509 → 8

Vemos que 2 clusters foram exlcuidos e alguns estão muito desbalanceados, portanto vamos precisar balancear para evitar overfit

# 3. Separação de dados

In [6]:
modeling_data = {}

for cl in unique_clusters:
    # Filtrando cluster desejado
    cl_offers = offer_for_modeling[offer_for_modeling["cluster"] == cl]

    cl_account_ids = cl_offers[partition_col].unique()

    # Separar dados de treino/teste por account_id
    train_ids, val_ids = train_test_split(cl_account_ids, test_size=0.3, random_state=3)

    # Dados de treino
    df_train = cl_offers[cl_offers["account_id"].isin(train_ids)].reset_index(drop=True)
    # Balanceando dados de treino
    target_count = df_train[df_train["cluster"] == cl][y_col[0]].value_counts().to_dict()
    small_cat = target_count[1] if target_count[1] < target_count[0] else target_count[0]
    df_train_bal = pd.concat(
        [df_train[df_train[y_col[0]] == 1].sample(small_cat, random_state=3),
         df_train[df_train[y_col[0]] == 0].sample(small_cat, random_state=3)]
    )
    # Dados de validação
    df_val = cl_offers[cl_offers["account_id"].isin(val_ids)].reset_index(drop=True)
    # Balanceando dados de validação
    target_count = df_val[df_val["cluster"] == cl][y_col[0]].value_counts().to_dict()
    small_cat = target_count[1] if target_count[1] < target_count[0] else target_count[0]
    df_val_bal = pd.concat(
        [df_val[df_val[y_col[0]] == 1].sample(small_cat, random_state=3),
         df_val[df_val[y_col[0]] == 0].sample(small_cat, random_state=3)]
    )

    print_info = {
        # Não balanceado
        "train_size":   f"{round(len(df_train)):,}",
        "val_size":     f"{round(len(df_val)):,}",
        "train_p":      f"{round(len(df_train)*100 / (len(df_train) + len(df_val))):,}%",
        "val_p":        f"{round(len(df_val)*100 / (len(df_train) + len(df_val))):,}%",
        "train_size_0": f"{round(len(df_train[df_train[y_col[0]] == 0])):,}",
        "train_size_1": f"{round(len(df_train[df_train[y_col[0]] == 1])):,}",
        "val_size_0":   f"{round(len(df_val[df_val[y_col[0]] == 0])):,}",
        "val_size_1":   f"{round(len(df_val[df_val[y_col[0]] == 1])):,}",
        # Balanceado
        "train_bal_size":   f"{round(len(df_train_bal)):,}",
        "val_bal_size":     f"{round(len(df_val_bal)):,}",
        "train_bal_p":      f"{round(len(df_train_bal)*100 / (len(df_train_bal) + len(df_val_bal))):,}%",
        "val_bal_p":        f"{round(len(df_val_bal)*100 / (len(df_train_bal) + len(df_val_bal))):,}%",
        "train_bal_size_0": f"{round(len(df_train_bal[df_train_bal[y_col[0]] == 0])):,}",
        "train_bal_size_1": f"{round(len(df_train_bal[df_train_bal[y_col[0]] == 1])):,}",
        "val_bal_size_0":   f"{round(len(df_val_bal[df_val_bal[y_col[0]] == 0])):,}",
        "val_bal_size_1":   f"{round(len(df_val_bal[df_val_bal[y_col[0]] == 1])):,}",
    }
    print(f"Cluster {cl} (ñ balanceado): Treino {print_info['train_size']} ({print_info['train_p']} → {print_info['train_size_1']} | {print_info['train_size_0']}) | Validação {print_info['val_size']} ({print_info['val_p']} → {print_info['val_size_1']} | {print_info['val_size_0']})")
    print(f"Cluster {cl}   (balanceado): Treino {print_info['train_bal_size']} ({print_info['train_bal_p']} → {print_info['train_bal_size_1']} | {print_info['train_bal_size_0']}) | Validação {print_info['val_bal_size']} ({print_info['val_bal_p']} → {print_info['val_bal_size_1']} | {print_info['val_bal_size_0']})")
    print()

    # Definindo X e y
    modeling_data[cl] = {
        "X_train": df_train[X_cols], # Com target não balanceada
        "y_train": df_train[y_col],
        "X_train_bal": df_train_bal[X_cols], # Com target balanceada
        "y_train_bal": df_train_bal[y_col],

        "X_val": df_val[X_cols], # Com target não balanceada
        "y_val": df_val[y_col],
        "X_val_bal": df_val_bal[X_cols], # Com target balanceada
        "y_val_bal": df_val_bal[y_col],

        "metrics": {},
        "metrics_full_table": {},
        "params": {},
        "model": None
    }


Cluster 1 (ñ balanceado): Treino 4,322 (69% → 3,976 | 346) | Validação 1,910 (31% → 1,747 | 163)
Cluster 1   (balanceado): Treino 692 (68% → 346 | 346) | Validação 326 (32% → 163 | 163)

Cluster 2 (ñ balanceado): Treino 5,627 (70% → 1,901 | 3,726) | Validação 2,389 (30% → 814 | 1,575)
Cluster 2   (balanceado): Treino 3,802 (70% → 1,901 | 1,901) | Validação 1,628 (30% → 814 | 814)

Cluster 4 (ñ balanceado): Treino 24,039 (70% → 22,319 | 1,720) | Validação 10,312 (30% → 9,631 | 681)
Cluster 4   (balanceado): Treino 3,440 (72% → 1,720 | 1,720) | Validação 1,362 (28% → 681 | 681)

Cluster 5 (ñ balanceado): Treino 5,858 (70% → 4,456 | 1,402) | Validação 2,465 (30% → 1,796 | 669)
Cluster 5   (balanceado): Treino 2,804 (68% → 1,402 | 1,402) | Validação 1,338 (32% → 669 | 669)

Cluster 6 (ñ balanceado): Treino 3,865 (71% → 1,177 | 2,688) | Validação 1,612 (29% → 508 | 1,104)
Cluster 6   (balanceado): Treino 2,354 (70% → 1,177 | 1,177) | Validação 1,016 (30% → 508 | 508)

Cluster 7 (ñ balancead

Com essa separação dos dados por account_id temos o beneficio de não correr o risco de ter um bom resultado por causa da similaridade de comportamento de um usuário que esta na base de treino e validação ao mesmo tempo.

Seria um problema utilizar essa abordagem de separação se os dados de treino e teste ficassem com tamanhos muito diferentes de 70/30, mas como vimos no print anterior, todos os clusters obtiveram uma boa separação aleatória.

# 4. Otimizações de hiperparametros

A métrica a ser utilizada é a precisão, isso porque queremos monitorar se o modelo esta prevendo corretamente quando ele diz que a oferta será utilizada.

Isso é importante pois queremos que ele detecte corretamente quais os tipos de oferta que devemos oferecer para cada cliente.

In [7]:
for cl in tqdm(unique_clusters[:], mininterval=10, ncols=100):
    # Gerando instancia inicial do modelo
    CBC = CatBoostClassifier(
        verbose=False,
        random_seed=3,
        cat_features=categorical_features,
        train_dir='tmp_catboost_info'
    )

    # Definindo parametros a serem testados
    param_dist = {
        "depth": [4, 6, 8],
        "learning_rate": [0.01, 0.05, 0.1, 0.2],
        "iterations": [100, 200, 500],
        "l2_leaf_reg": [1, 3, 5, 7, 9],
        "border_count": [32, 64, 128]
    }

    # Realizando testes aleatórios para detectar o mlehor resultado
    random_search = RandomizedSearchCV(
        CBC,
        param_distributions=param_dist,
        n_iter=15, # quantas combinações testar
        scoring="precision", # "accuracy", "precision"
        cv=6, # Quantas vezes rodar validação cruzada
        verbose=False,
        n_jobs=-1,
        random_state=3,
    )

    # Treinamento
    random_search.fit(
        modeling_data[cl]["X_train_bal"],
        modeling_data[cl]["y_train_bal"]
    )

    # Melhor modelo
    modeling_data[cl]["params"] = random_search.best_params_
    modeling_data[cl]["model"] = random_search.best_estimator_

    # Gerando demais métricas para o melhor modelo
    pred_val = modeling_data[cl]["model"].predict(modeling_data[cl]["X_val_bal"])
    modeling_data[cl]["metrics"] = {
        "precision": precision_score(modeling_data[cl]["y_val_bal"], pred_val),
        "accuracy": accuracy_score(modeling_data[cl]["y_val_bal"], pred_val),
        "recall": recall_score(modeling_data[cl]["y_val_bal"], pred_val),
        "f1": f1_score(modeling_data[cl]["y_val_bal"], pred_val),
        "log_loss": log_loss(modeling_data[cl]["y_val_bal"], pred_val),
    }

    try:
        MODEL = Model(modeling_data[cl])
        dump(MODEL, f"../src/models/catboost_cl{cl}.joblib")
    except:
        os.mkdir("../src/models")
        MODEL = Model(modeling_data[cl])
        dump(MODEL, f"../src/models/catboost_cl{cl}.joblib")

100%|████████████████████████████████████████████████████████████████| 6/6 [31:29<00:00, 314.89s/it]


# 5. Analisando as métricas dos modelos

In [8]:
for cl in unique_clusters:
    print(f"Analisando modelo do cluster {cl}")
    print(f'- Precisão: {round(modeling_data[cl]["metrics"]["precision"]*100)}%')
    print(f'- Acurácia: {round(modeling_data[cl]["metrics"]["accuracy"]*100)}%')
    print(f'- Recall: {round(modeling_data[cl]["metrics"]["recall"]*100)}%')
    print(f'- F1: {round(modeling_data[cl]["metrics"]["f1"]*100)}%')
    print("\n")

Analisando modelo do cluster 1
- Precisão: 99%
- Acurácia: 94%
- Recall: 89%
- F1: 94%


Analisando modelo do cluster 2
- Precisão: 90%
- Acurácia: 92%
- Recall: 95%
- F1: 92%


Analisando modelo do cluster 4
- Precisão: 92%
- Acurácia: 92%
- Recall: 92%
- F1: 92%


Analisando modelo do cluster 5
- Precisão: 93%
- Acurácia: 91%
- Recall: 88%
- F1: 91%


Analisando modelo do cluster 6
- Precisão: 93%
- Acurácia: 95%
- Recall: 96%
- F1: 95%


Analisando modelo do cluster 7
- Precisão: 94%
- Acurácia: 94%
- Recall: 94%
- F1: 94%




# 7. Desligando PC

In [ ]:
# from sleep import sleep
# sleep(20)

# import os
# os.system("shutdown /s /t 0")

ModuleNotFoundError: No module named 'sleep'